

https://colab.research.google.com/drive/1wfBW4ekWIkCpmL411x2mTg-UzE3nGJE_?usp=sharing#scrollTo=qTCAWOV-YSDw



In [1]:
#necessary libraries
import re
import chardet 
import operator
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from wordcloud import WordCloud

from nltk import pos_tag
from nltk.corpus import wordnet
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from keras.layers.normalization import BatchNormalization
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import LSTM, Dense, Bidirectional
from tensorflow.keras.layers import Dropout, SpatialDropout1D

from tqdm import tqdm
from wordcloud import WordCloud, STOPWORDS
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
data = pd.read_csv('/content/train.csv')

In [3]:
data_test = pd.read_csv('/content/test.csv')

In [4]:
data_test.head()

ID  ... Sub-task B
0    C7.2589  ...       NGEN
1    C68.872  ...       NGEN
2    C36.762  ...        GEN
3  C4.1540.1  ...       NGEN
4     C59.68  ...       NGEN

[5 rows x 4 columns]

In [5]:
data.drop(['ID','Sub-task B'], axis=1, inplace=True)

In [6]:
data_test.drop(['ID','Sub-task B'], axis=1, inplace=True)


In [7]:
data.head()

Text Sub-task A
0                                          Next part        NAG
1                 Iii8mllllllm\nMdxfvb8o90lplppi0005        NAG
2  🤣🤣😂😂🤣🤣🤣😂osm vedio ....keep it up...make more v...        NAG
3  What the fuck was this? I respect shwetabh and...        NAG
4  Concerned authorities should bring arundathi R...        NAG

In [8]:
data['Sub-task A'].value_counts()

NAG    3375
CAG     453
OAG     435
Name: Sub-task A, dtype: int64

In [9]:
NAG_train = data[(data['Sub-task A']=='NAG')]
OAG_train = data[(data['Sub-task A']=='CAG')]
CAG_train = data[(data['Sub-task A']=='OAG')]

In [10]:
NAG = data_test[(data_test['Sub-task A']=='NAG')]
OAG = data_test[(data_test['Sub-task A']=='CAG')]
CAG = data_test[(data_test['Sub-task A']=='OAG')]

In [11]:
import warnings as wrn
wrn.filterwarnings('ignore')

NAG_train["Sub-task A"] = 0 
#negatives_test["Sentiment"] = 0

OAG_train["Sub-task A"] = 1
#positives_test["Sentiment"] = 2

CAG_train["Sub-task A"] = 2
#neutrals_test["Sentiment"] = 1

NAG_train.head()

Text  Sub-task A
0                                          Next part           0
1                 Iii8mllllllm\nMdxfvb8o90lplppi0005           0
2  🤣🤣😂😂🤣🤣🤣😂osm vedio ....keep it up...make more v...           0
3  What the fuck was this? I respect shwetabh and...           0
4  Concerned authorities should bring arundathi R...           0

In [12]:
import warnings as wrn
wrn.filterwarnings('ignore')

NAG["Sub-task A"] = 0 
#negatives_test["Sentiment"] = 0

OAG["Sub-task A"] = 1
#positives_test["Sentiment"] = 2

CAG["Sub-task A"] = 2
#neutrals_test["Sentiment"] = 1

NAG.head()

Text  Sub-task A
0        U deserve more subscribers. U really great.           0
1                                     Nice video....           0
2  sorry if i bother somebody.. iam a defence asp...           0
3  Joker was amazing....it was not glamorised !.....           0
4                                          Nice baro           0

In [13]:
data_train = pd.concat([NAG_train,
                  OAG_train,
                  CAG_train
                 ],axis=0)


In [14]:
data_test = pd.concat([NAG,
                  OAG,
                  CAG
                 ],axis=0)


In [15]:

import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
cleanedData = []
texts = []

lemma = WordNetLemmatizer()
swords = stopwords.words("english")
for text in data_train["Text"]:
    
    # Cleaning links
    text = re.sub(r'http\S+', '', text)
    
    # Cleaning everything except alphabetical and numerical characters
    text = re.sub("[^a-zA-Z0-9]"," ",text)
    
    # Tokenizing and lemmatizing
    text = nltk.word_tokenize(text.lower())
    text = [lemma.lemmatize(word) for word in text]
    
    # Removing stopwords
    text = [word for word in text if word not in swords]
    texts.append(text)
    # Joining
    text = " ".join(text)
    
    cleanedData.append(text)

In [16]:
from gensim.models import  Word2Vec

In [17]:
## for data
import json
import pandas as pd
import numpy as np
## for plotting
import matplotlib.pyplot as plt
import seaborn as sns
## for processing
import re
import nltk
## for bag-of-words
from sklearn import feature_extraction, model_selection, naive_bayes, pipeline, manifold, preprocessing
## for explainer
#from lime import lime_text
## for word embedding
import gensim
import gensim.downloader as gensim_api
## for deep learning
from tensorflow.keras import models, layers, preprocessing as kprocessing
from tensorflow.keras import backend as K
## for bert language model
#import transformers

In [18]:
!python -m spacy download en_core_web_lg

     |████████████████████████████████| 827.9MB 1.2MB/s 
  Created wheel for en-core-web-lg: filename=en_core_web_lg-2.2.5-cp37-none-any.whl size=829180944 sha256=7c9c24e1d56f14dfbe886e31a652270dde33584f4cb0737ee3ce10cbfd41ac15
  Stored in directory: /tmp/pip-ephem-wheel-cache-ghe_t1r7/wheels/2a/c1/a6/fc7a877b1efca9bc6a089d6f506f16d3868408f9ff89f8dbfc
Successfully built en-core-web-lg
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


In [19]:
import en_core_web_lg
nlp = en_core_web_lg.load()

In [20]:
import numpy as np
import pandas as pd

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

from sklearn.metrics import classification_report

In [21]:
def get_vec(x):
  doc = nlp(x)
  vec = doc.vector
  return vec

In [23]:
data_train['vec'] = data_train['Text'].apply(lambda x: get_vec(x))

In [24]:
data_test['vec'] = data_test['Text'].apply(lambda x: get_vec(x))

In [25]:
X = data_train['vec'].to_numpy()
X = X.reshape(-1, 1)

In [27]:
P = data_test['vec'].to_numpy()
P = P.reshape(-1, 1)

In [28]:
X = np.concatenate(np.concatenate(X, axis = 0), axis = 0).reshape(-1, 300)

In [29]:
P = np.concatenate(np.concatenate(P, axis = 0), axis = 0).reshape(-1, 300)

In [30]:
X.shape

(4263, 300)

In [31]:
y = data_train['Sub-task A']

In [32]:
Q = data_test['Sub-task A']

In [33]:
X_train = X
X_test = P
y_train = y 
y_test = Q



In [ ]:
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0, stratify = y)

In [34]:
X_train.shape, X_test.shape

((4263, 300), (1066, 300))

In [35]:
clf = LogisticRegression(solver='liblinear')

In [36]:
clf.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)

In [37]:
y_pred = clf.predict(X_test)

In [38]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.80      0.96      0.87       836
           1       0.09      0.01      0.02       117
           2       0.43      0.20      0.28       113

    accuracy                           0.78      1066
   macro avg       0.44      0.39      0.39      1066
weighted avg       0.68      0.78      0.72      1066

